In [13]:
from tools import get_puzzle, show_problem_1, show_problem_2

TODAY = 22
puzzle = get_puzzle(TODAY)
show_problem_1(puzzle)

https://adventofcode.com/2023/day/22
## --- Day 22: Sand Slabs ---


Enough sand has fallen; it can finally filter water for Snow Island.


Well, **almost** .


The sand has been falling as large compacted **bricks** of sand, piling up to form an impressive stack here near the edge of Island Island. In order to make use of the sand to filter water, some of the bricks will need to be broken apart - nay, **disintegrated** - back into freely flowing sand.


The stack is tall enough that you'll have to be careful about choosing which bricks to disintegrate; if you disintegrate the wrong brick, large portions of the stack could topple, which sounds pretty dangerous.


The Elves responsible for water filtering operations took a **snapshot of the bricks while they were still falling** (your puzzle input) which should let you work out which bricks are safe to disintegrate. For example:


```
 1,0,1~1,2,1
 0,0,2~2,0,2
 0,2,3~2,2,3
 0,0,4~0,2,4
 2,0,5~2,2,5
 0,1,6~2,1,6
 1,1,8~1,1,9

```


Each line of text in the snapshot represents the position of a single brick at the time the snapshot was taken. The position is given as twox,y,z``coordinates - one for each end of the brick - separated by a tilde (~``). Each brick is made up of a single straight line of cubes, and the Elves were even careful to choose a time for the snapshot that had all of the free-falling bricks at **integer positions above the ground** , so the whole snapshot is aligned to a three-dimensional cube grid.


A line like2,2,2~2,2,2``means that both ends of the brick are at the same coordinate - in other words, that the brick is a single cube.


Lines like0,0,10~1,0,10``or0,0,10~0,1,10``both represent bricks that are **two cubes** in volume, both oriented horizontally. The first brick extends in thex``direction, while the second brick extends in they``direction.


A line like0,0,1~0,0,10``represents a **ten-cube brick** which is oriented **vertically** . One end of the brick is the cube located at0,0,1``, while the other end of the brick is located directly above it at0,0,10``.


The ground is atz=0``and is perfectly flat; the lowestz``value a brick can have is therefore1``. So,5,5,1~5,6,1``and0,2,1~0,2,5``are both resting on the ground, but3,3,2~3,3,3``was above the ground at the time of the snapshot.


Because the snapshot was taken while the bricks were still falling, some bricks will **still be in the air** ; you'll need to start by figuring out where they will end up. Bricks are magically stabilized, so they **never rotate** , even in weird situations like where a long horizontal brick is only supported on one end. Two bricks cannot occupy the same position, so a falling brick will come to rest upon the first other brick it encounters.


Here is the same example again, this time with each brick given a letter so it can be marked in diagrams:


```
 1,0,1~1,2,1   <- A
 0,0,2~2,0,2   <- B
 0,2,3~2,2,3   <- C
 0,0,4~0,2,4   <- D
 2,0,5~2,2,5   <- E
 0,1,6~2,1,6   <- F
 1,1,8~1,1,9   <- G

```


At the time of the snapshot, from the side so thex``axis goes left to right, these bricks are arranged like this:


```
  x
 012
 .G. 9
 .G. 8
 ... 7
 FFF 6
 ..E 5 z
 D.. 4
 CCC 3
 BBB 2
 .A. 1
 --- 0

```


Rotating the perspective 90 degrees so they``axis now goes left to right, the same bricks are arranged like this:


```
  y
 012
 .G. 9
 .G. 8
 ... 7
 .F. 6
 EEE 5 z
 DDD 4
 ..C 3
 B.. 2
 AAA 1
 --- 0

```


Once all of the bricks fall downward as far as they can go, the stack looks like this, where?``means bricks are hidden behind other bricks at that location:


```
  x
 012
 .G. 6
 .G. 5
 FFF 4
 D.E 3 z
 ??? 2
 .A. 1
 --- 0

```


Again from the side:


```
  y
 012
 .G. 6
 .G. 5
 .F. 4
 ??? 3 z
 B.C 2
 AAA 1
 --- 0

```


Now that all of the bricks have settled, it becomes easier to tell which bricks are supporting which other bricks:


+ BrickA``is the only brick supporting bricksB``andC``.
+ BrickB``is one of two bricks supporting brickD``and brickE``.
+ BrickC``is the other brick supporting brickD``and brickE``.
+ BrickD``supports brickF``.
+ BrickE``also supports brickF``.
+ BrickF``supports brickG``.
+ BrickG``isn't supporting any bricks.


Your first task is to figure out **which bricks are safe to disintegrate** . A brick can be safely disintegrated if, after removing it, **no other bricks** would fall further directly downward. Don't actually disintegrate any bricks - just determine what would happen if, for each brick, only that brick were disintegrated. Bricks can be disintegrated even if they're completely surrounded by other bricks; you can squeeze between bricks if you need to.


In this example, the bricks can be disintegrated as follows:


+ BrickA``cannot be disintegrated safely; if it were disintegrated, bricksB``andC``would both fall.
+ BrickB`` **can** be disintegrated; the bricks above it (D``andE``) would still be supported by brickC``.
+ BrickC`` **can** be disintegrated; the bricks above it (D``andE``) would still be supported by brickB``.
+ BrickD`` **can** be disintegrated; the brick above it (F``) would still be supported by brickE``.
+ BrickE`` **can** be disintegrated; the brick above it (F``) would still be supported by brickD``.
+ BrickF``cannot be disintegrated; the brick above it (G``) would fall.
+ BrickG`` **can** be disintegrated; it does not support any other bricks.


So, in this example,5``bricks can be safely disintegrated.


Figure how the blocks will settle based on the snapshot. Once they've settled, consider disintegrating a single brick; **how many bricks could be safely chosen as the one to get disintegrated?** 




In [26]:
import queue

ON = HIGH = 1
OFF = LOW = 0
DEBUG = False
class Brick(object):
    def __init__(self, data):
        self.s = data[0]  # start
        self.e = data[1]  # end
    
    def get_lowest_squares (self):
        squares = []
        if self.s[2] == self.e[2]:
            pass
        elif self.s[2] > self.e[2]:
            yield [self.e[0],self.e[1], self.e[2]]
        else:
            yield [self.s[0],self.s[1], self.s[2]]


    def __repr__(self):
        return str(self.s) + "-" + str(self.e) + "\n" 

def build_state(data):
    state = []
    for line in data:
        state.append(Brick([list(map(int, x.split(","))) for x in line.split("~")]))
    return state

def solution_1(data):

    state = build_state(data)
    print(state)
   
    
    return 5


assert solution_1 (puzzle.test) == 5
#print( f"Solution 1:  {solution_1 (puzzle.data)} is the product of the number of low pulses times the number of high pulses sent ")




[[1, 0, 1]-[1, 2, 1]
, [0, 0, 2]-[2, 0, 2]
, [0, 2, 3]-[2, 2, 3]
, [0, 0, 4]-[0, 2, 4]
, [2, 0, 5]-[2, 2, 5]
, [0, 1, 6]-[2, 1, 6]
, [1, 1, 8]-[1, 1, 9]
]


In [28]:
def test ():
    yield from range(4)

In [29]:
print (test())

<generator object test at 0x7a4b4eeda120>


In [33]:
a = [1,2,3,4,5,6,7,8,9,0]

print (a[3:5])

[4, 5]
